In [1]:
!pip install git+https://github.com/ildoonet/pytorch-gradual-warmup-lr.git
!git clone https://github.com/haqishen/SIIM-ISIC-Melanoma-Classification-1st-Place-Solution.git
%cd SIIM-ISIC-Melanoma-Classification-1st-Place-Solution
!git clone https://github.com/vcadillog/Ensemble-Deep-Learning-Melanoma-Competition-Pytorch    
!pip install geffnet
!pip install git+https://github.com/zhanghang1989/ResNeSt
!pip install pretrainedmodels

  Cloning https://github.com/ildoonet/pytorch-gradual-warmup-lr.git to /tmp/pip-req-build-ayacbqoq
  Running command git clone --filter=blob:none --quiet https://github.com/ildoonet/pytorch-gradual-warmup-lr.git /tmp/pip-req-build-ayacbqoq
  Resolved https://github.com/ildoonet/pytorch-gradual-warmup-lr.git to commit 6b5e8953a80aef5b324104dc0c2e9b8c34d622bd
  Preparing metadata (setup.py) ... done
  Created wheel for warmup-scheduler: filename=warmup_scheduler-0.3.2-py3-none-any.whl size=3880 sha256=ab54fe5fdc80b0ce687bff2e6f3e75a45b3a2699b7d7f7851a470f4b191546f4
  Stored in directory: /tmp/pip-ephem-wheel-cache-_tpl3z0t/wheels/bf/81/52/0e3bc0b645a339f94c76b4dcb8c8b7a5f588a614f5add83b9f
Successfully built warmup-scheduler
Cloning into 'SIIM-ISIC-Melanoma-Classification-1st-Place-Solution'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 39 (delta 17), reused 19 (delta 4), pack-reused 0

In [7]:
import os
import time
import random
import argparse
import numpy as np
import pandas as pd
from tqdm import tqdm
import cv2
import PIL.Image

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data.sampler import RandomSampler, SequentialSampler
from torch.optim.lr_scheduler import CosineAnnealingLR
from util import GradualWarmupSchedulerV2

from dataset import get_df, get_transforms, MelanomaDataset
from models import Effnet_Melanoma, Resnest_Melanoma, Seresnext_Melanoma


def set_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True


def train_epoch(model, loader, optimizer):

    model.train()
    train_loss = []
    bar = tqdm(loader)
    for (data, target) in bar:

        optimizer.zero_grad()
        
        if args.use_meta:
            data, meta = data
            data, meta, target = data.to(device), meta.to(device), target.to(device)
            logits = model(data, meta)
        else:
            data, target = data.to(device), target.to(device)
            logits = model(data)        
        
        loss = criterion(logits, target)

        if not args.use_amp:
            loss.backward()

        if args.image_size in [896,576]:
            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        loss_np = loss.detach().cpu().numpy()
        train_loss.append(loss_np)
        smooth_loss = sum(train_loss[-100:]) / min(len(train_loss), 100)
        bar.set_description('loss: %.5f, smth: %.5f' % (loss_np, smooth_loss))

    train_loss = np.mean(train_loss)
    return train_loss


def get_trans(img, I):

    if I >= 4:
        img = img.transpose(2, 3)
    if I % 4 == 0:
        return img
    elif I % 4 == 1:
        return img.flip(2)
    elif I % 4 == 2:
        return img.flip(3)
    elif I % 4 == 3:
        return img.flip(2).flip(3)


def val_epoch(model, loader, mel_idx, is_ext=None, n_test=1, get_output=False):

    model.eval()
    val_loss = []
    LOGITS = []
    PROBS = []
    TARGETS = []
    with torch.no_grad():
        for (data, target) in tqdm(loader):
            
            if args.use_meta:
                data, meta = data
                data, meta, target = data.to(device), meta.to(device), target.to(device)
                logits = torch.zeros((data.shape[0], args.out_dim)).to(device)
                probs = torch.zeros((data.shape[0], args.out_dim)).to(device)
                for I in range(n_test):
                    l = model(get_trans(data, I), meta)
                    logits += l
                    probs += l.softmax(1)
            else:
                data, target = data.to(device), target.to(device)
                logits = torch.zeros((data.shape[0], args.out_dim)).to(device)
                probs = torch.zeros((data.shape[0], args.out_dim)).to(device)
                for I in range(n_test):
                    l = model(get_trans(data, I))
                    logits += l
                    probs += l.softmax(1)
            logits /= n_test
            probs /= n_test

            LOGITS.append(logits.detach().cpu())
            PROBS.append(probs.detach().cpu())
            TARGETS.append(target.detach().cpu())

            loss = criterion(logits, target)
            val_loss.append(loss.detach().cpu().numpy())

    val_loss = np.mean(val_loss)
    LOGITS = torch.cat(LOGITS).numpy()
    PROBS = torch.cat(PROBS).numpy()
    TARGETS = torch.cat(TARGETS).numpy()

    if get_output:
        return LOGITS, PROBS
    else:
        acc = (PROBS.argmax(1) == TARGETS).mean() * 100.
        auc = roc_auc_score((TARGETS == mel_idx).astype(float), PROBS[:, mel_idx])
        auc_20 = roc_auc_score((TARGETS[is_ext == 0] == mel_idx).astype(float), PROBS[is_ext == 0, mel_idx])
        return val_loss, acc, auc, auc_20


def run(fold, df, meta_features, n_meta_features, transforms_train, transforms_val, mel_idx):

    if args.DEBUG:
        args.n_epochs = 5
        df_train = df[df['fold'] != fold].sample(args.batch_size * 5)
        df_valid = df[df['fold'] == fold].sample(args.batch_size * 5)
    else:
        df_train = df[df['fold'] != fold]
        df_valid = df[df['fold'] == fold]

    dataset_train = MelanomaDataset(df_train, 'train', meta_features, transform=transforms_train)
    dataset_valid = MelanomaDataset(df_valid, 'valid', meta_features, transform=transforms_val)
    train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=args.batch_size, sampler=RandomSampler(dataset_train), num_workers=args.num_workers)
    valid_loader = torch.utils.data.DataLoader(dataset_valid, batch_size=args.batch_size, num_workers=args.num_workers)

    model = ModelClass(
        args.enet_type,
        n_meta_features=n_meta_features,
        n_meta_dim=[int(nd) for nd in args.n_meta_dim.split(',')],
        out_dim=args.out_dim,
        pretrained=True
    )
          
    model = model.to(device)
    if args.wandb:
        wandb.watch(model)

    auc_max = 0.
    auc_20_max = 0.
    model_file  = os.path.join(args.model_dir, f'{args.kernel_type}_best_fold{fold}.pth')
    model_file2 = os.path.join(args.model_dir, f'{args.kernel_type}_best_20_fold{fold}.pth')
    model_file3 = os.path.join(args.model_dir, f'{args.kernel_type}_final_fold{fold}.pth')

    optimizer = optim.Adam(model.parameters(), lr=args.init_lr)

    if DP:
        model = nn.DataParallel(model)

    scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, args.n_epochs - 1)
    scheduler_warmup = GradualWarmupSchedulerV2(optimizer, multiplier=10, total_epoch=1, after_scheduler=scheduler_cosine)
    
    print(len(dataset_train), len(dataset_valid))

    for epoch in range(1, args.n_epochs + 1):
        print(time.ctime(), f'Fold {fold}, Epoch {epoch}')


        train_loss = train_epoch(model, train_loader, optimizer)
        val_loss, acc, auc, auc_20 = val_epoch(model, valid_loader, mel_idx, is_ext=df_valid['is_ext'].values)
        if args.wandb:
            wandb.log(
                {"Epoch" : epoch,
                 "Fold" : fold,
                 "Train Loss": train_loss,
                 "Validation Loss": val_loss,
                 "Accuracy": acc
                })            

        content = time.ctime() + ' ' + f'Fold {fold}, Epoch {epoch}, lr: {optimizer.param_groups[0]["lr"]:.7f}, train loss: {train_loss:.5f}, valid loss: {(val_loss):.5f}, acc: {(acc):.4f}, auc: {(auc):.6f}, auc_20: {(auc_20):.6f}.'
        print(content)
        with open(os.path.join(args.log_dir, f'log_{args.kernel_type}.txt'), 'a') as appender:
            appender.write(content + '\n')

        scheduler_warmup.step()    
        if epoch==2: scheduler_warmup.step() # bug workaround   
            
        if auc > auc_max:
            print('auc_max ({:.6f} --> {:.6f}). Saving model ...'.format(auc_max, auc))
            torch.save(model.state_dict(), model_file)
            auc_max = auc
        if auc_20 > auc_20_max:
            print('auc_20_max ({:.6f} --> {:.6f}). Saving model ...'.format(auc_20_max, auc_20))
            torch.save(model.state_dict(), model_file2)
            auc_20_max = auc_20

    torch.save(model.state_dict(), model_file3)


def train():
    
    df, df_test, meta_features, n_meta_features, mel_idx = get_df(
        args.kernel_type,
        args.out_dim,
        args.data_dir,
        args.data_folder,
        args.use_meta
    )

    transforms_train, transforms_val = get_transforms(args.image_size)
    
    print('Data preprocessed')

    folds = [int(i) for i in args.fold.split(',')]
    for fold in folds:
        run(fold, df, meta_features, n_meta_features, transforms_train, transforms_val, mel_idx)

def predict():

    df, df_test, meta_features, n_meta_features, mel_idx = get_df(
        args.kernel_type,
        args.out_dim,
        args.data_dir,
        args.data_folder,
        args.use_meta
    )

    transforms_train, transforms_val = get_transforms(args.image_size)

    if args.DEBUG:
        df_test = df_test.sample(args.batch_size * 3)
    dataset_test = MelanomaDataset(df_test, 'test', meta_features, transform=transforms_val)
    test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=args.batch_size, num_workers=args.num_workers)

    # load model
    models = []
    folds = [int(i) for i in args.fold.split(',')]
    for fold in folds:
#     for fold in range(5):

        if args.eval == 'best':
            model_file = os.path.join(args.model_dir, f'{args.kernel_type}_best_fold{fold}.pth')
        elif args.eval == 'best_20':
            model_file = os.path.join(args.model_dir, f'{args.kernel_type}_best_20_fold{fold}.pth')
        if args.eval == 'final':
            model_file = os.path.join(args.model_dir, f'{args.kernel_type}_final_fold{fold}.pth')

        model = ModelClass(
            args.enet_type,
            n_meta_features=n_meta_features,
            n_meta_dim=[int(nd) for nd in args.n_meta_dim.split(',')],
            out_dim=args.out_dim
        )
        model = model.to(device)

        try:  # single GPU model_file
            model.load_state_dict(torch.load(model_file), strict=True)
        except:  # multi GPU model_file
            state_dict = torch.load(model_file)
            state_dict = {k[7:] if k.startswith('module.') else k: state_dict[k] for k in state_dict.keys()}
            model.load_state_dict(state_dict, strict=True)
        
        if len(os.environ['CUDA_VISIBLE_DEVICES']) > 1:
            model = torch.nn.DataParallel(model)

        model.eval()
        models.append(model)

    # predict
    PROBS = []
    with torch.no_grad():
        for (data) in tqdm(test_loader):
            if args.use_meta:
                data, meta = data
                data, meta = data.to(device), meta.to(device)
                probs = torch.zeros((data.shape[0], args.out_dim)).to(device)
                for model in models:
                    for I in range(args.n_test):
                        l = model(get_trans(data, I), meta)
                        probs += l.softmax(1)
            else:   
                data = data.to(device)
                probs = torch.zeros((data.shape[0], args.out_dim)).to(device)
                for model in models:
                    for I in range(args.n_test):
                        l = model(get_trans(data, I))
                        probs += l.softmax(1)

            probs /= args.n_test
            probs /= len(models)

            PROBS.append(probs.detach().cpu())

    PROBS = torch.cat(PROBS).numpy()

    # save cvs
    df_test['target'] = PROBS[:, mel_idx]
    df_test[['image_name', 'target']].to_csv(os.path.join(args.sub_dir, f'sub_{args.kernel_type}_{args.eval}.csv'), index=False)



In [8]:
class Args():
    def __init__(self, mode = 'train'):
        assert mode in ['train','predict']
        self.kernel_type = 'newfold'
        self.data_dir = '/kaggle/input/'
        self.data_folder = 256
        self.image_size = 256
        self.enet_type = 'efficientnet_b3'
        self.batch_size = 64
        self.num_workers = 2
        self.init_lr = 3e-5
        self.out_dim = 9
        self.n_epochs = 15
        self.use_amp = False
        self.use_meta = True
        self.DEBUG = False #True to test the code
        
        self.model_dir = './Ensemble-Deep-Learning-Melanoma-Competition-Pytorch/weights/weights_nometa'  #Work in github folder
#         self.model_dir = './weights'        
        self.log_dir = './logs'
        self.CUDA_VISIBLE_DEVICES = '0,1'
#         self.fold = '0,1,2,3,4'
        self.fold = '0,1' # for demonstration purposes only trained on 2 folds       #    
        self.wandb = False # Turn True if you wanna use your W&B account
        if mode == 'predict':            
            self.eval = 'best'
            self.sub_dir = './Ensemble-Deep-Learning-Melanoma-Competition-Pytorch/subs/subs_nometa'   #Saved model in github folder        
#             self.sub_dir = './subs'
            self.n_test = 8
#             self.DEBUG = False # 
        self.n_meta_dim = '512,128'
    


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
import wandb
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

# My API token is saved as a label named "wandb_api". 
# I strongly suggest you to not copy your API tokens directly in your notebook
wandb_api = user_secrets.get_secret('wandb_api') 

wandb.login(key=wandb_api)
wandb.init(name='Ensemble')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [9]:
#TRAINING ENSEMBLE        
args = Args('train')

os.makedirs(args.model_dir, exist_ok=True)
os.makedirs(args.log_dir, exist_ok=True)
os.environ['CUDA_VISIBLE_DEVICES'] = args.CUDA_VISIBLE_DEVICES

if args.enet_type == 'resnest101':
    ModelClass = Resnest_Melanoma
elif args.enet_type == 'seresnext101':
    ModelClass = Seresnext_Melanoma
elif 'efficientnet' in args.enet_type:
    ModelClass = Effnet_Melanoma
else:
    raise NotImplementedError()

DP = len(os.environ['CUDA_VISIBLE_DEVICES']) > 1

set_seed()

criterion = nn.CrossEntropyLoss()

train()


100%|██████████| 10982/10982 [00:35<00:00, 307.06it/s]
/opt/conda/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:1151: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:1177: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/albumentations/augmentations/blur/transforms.py:185: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  "blur_limit and sigma_limit minimum value can not be both equal to 0. "
/opt/conda/lib/python3.7/site-packages/albumentations/augmentations/dropout/cutout.py:51: FutureWarning: Cutout has been deprecated. Please use CoarseDropout
  FutureWarning,


Data preprocessed


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b3_ra2-cf984f9c.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b3_ra2-cf984f9c.pth


46382 11582
Wed Mar 15 01:26:31 2023 Fold 0, Epoch 1


100%|██████████| 181/181 [01:34<00:00,  1.91it/s]


...

Wed Mar 15 11:09:33 2023 Fold 1, Epoch 13, lr: 0.0000327, train loss: 0.23909, valid loss: 0.29744, acc: 90.5686, auc: 0.964711, auc_20: 0.923313.
auc_max (0.962204 --> 0.964711). Saving model ...
Wed Mar 15 11:09:34 2023 Fold 1, Epoch 14


100%|██████████| 182/182 [01:07<00:00,  2.71it/s]


Wed Mar 15 11:30:09 2023 Fold 1, Epoch 14, lr: 0.0000149, train loss: 0.22847, valid loss: 0.30161, acc: 90.6808, auc: 0.964675, auc_20: 0.924440.
Wed Mar 15 11:30:09 2023 Fold 1, Epoch 15


100%|██████████| 182/182 [01:07<00:00,  2.72it/s]


Wed Mar 15 11:50:50 2023 Fold 1, Epoch 15, lr: 0.0000038, train loss: 0.22321, valid loss: 0.30123, acc: 90.5341, auc: 0.964595, auc_20: 0.923668.


In [35]:
#PREDICT ENSEMBLE
args = Args('predict') 

os.makedirs(args.sub_dir, exist_ok=True)
os.environ['CUDA_VISIBLE_DEVICES'] = args.CUDA_VISIBLE_DEVICES

if args.enet_type == 'resnest101':
    ModelClass = Resnest_Melanoma
elif args.enet_type == 'seresnext101':
    ModelClass = Seresnext_Melanoma
elif 'efficientnet' in args.enet_type:
    ModelClass = Effnet_Melanoma
else:
    raise NotImplementedError()

DP = len(os.environ['CUDA_VISIBLE_DEVICES']) > 1

predict()    

100%|██████████| 10982/10982 [00:25<00:00, 434.86it/s]
/opt/conda/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:1151: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:1177: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/albumentations/augmentations/blur/transforms.py:185: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  "blur_limit and sigma_limit minimum value can not be both equal to 0. "
/opt/conda/lib/python3.7/site-packages/albumentations/augmentations/dropout/cutout.py:51: FutureWarning: Cutout has been deprecated. Please use CoarseDropout
  FutureWarning,
100%|██████████| 172/172 [06:01<00:00,  2.10s/it]


In [8]:
wandb.init(name='NoMeta')

wandb: Currently logged in as: vcadillo. Use `wandb login --relogin` to force relogin


In [13]:
args = Args('predict')
args.use_meta = False
args.model_dir = './Ensemble-Deep-Learning-Melanoma-Competition-Pytorch/weights/weights_nometa'


os.makedirs(args.model_dir, exist_ok=True)
os.makedirs(args.log_dir, exist_ok=True)
os.environ['CUDA_VISIBLE_DEVICES'] = args.CUDA_VISIBLE_DEVICES

if args.enet_type == 'resnest101':
    ModelClass = Resnest_Melanoma
elif args.enet_type == 'seresnext101':
    ModelClass = Seresnext_Melanoma
elif 'efficientnet' in args.enet_type:
    ModelClass = Effnet_Melanoma
else:
    raise NotImplementedError()

DP = len(os.environ['CUDA_VISIBLE_DEVICES']) > 1

set_seed()

criterion = nn.CrossEntropyLoss()

train()


/opt/conda/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:1151: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:1177: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/albumentations/augmentations/blur/transforms.py:185: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  "blur_limit and sigma_limit minimum value can not be both equal to 0. "
/opt/conda/lib/python3.7/site-packages/albumentations/augmentations/dropout/cutout.py:51: FutureWarning: Cutout has been deprecated. Please use CoarseDropout
  FutureWarning,


Data preprocessed


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b3_ra2-cf984f9c.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b3_ra2-cf984f9c.pth


46382 11582
Wed Mar 15 12:44:53 2023 Fold 0, Epoch 1


100%|██████████| 181/181 [01:30<00:00,  2.01it/s]


...

Wed Mar 15 21:57:25 2023 Fold 1, Epoch 13, lr: 0.0000327, train loss: 0.28558, valid loss: 0.34483, acc: 88.9464, auc: 0.958331, auc_20: 0.922056.
auc_max (0.957738 --> 0.958331). Saving model ...
Wed Mar 15 21:57:25 2023 Fold 1, Epoch 14


100%|██████████| 182/182 [00:52<00:00,  3.49it/s]


Wed Mar 15 22:16:40 2023 Fold 1, Epoch 14, lr: 0.0000149, train loss: 0.27445, valid loss: 0.34463, acc: 89.1880, auc: 0.957855, auc_20: 0.924843.
Wed Mar 15 22:16:40 2023 Fold 1, Epoch 15


100%|██████████| 182/182 [00:52<00:00,  3.48it/s]


Wed Mar 15 22:36:05 2023 Fold 1, Epoch 15, lr: 0.0000038, train loss: 0.27256, valid loss: 0.34145, acc: 89.2398, auc: 0.958082, auc_20: 0.922443.


In [18]:
args = Args('predict')
args.use_meta = False
args.model_dir = './Ensemble-Deep-Learning-Melanoma-Competition-Pytorch/weights/weights_nometa'
args.sub_dir = './Ensemble-Deep-Learning-Melanoma-Competition-Pytorch/subs/subs_nometa'

os.makedirs(args.sub_dir, exist_ok=True)
os.environ['CUDA_VISIBLE_DEVICES'] = args.CUDA_VISIBLE_DEVICES

if args.enet_type == 'resnest101':
    ModelClass = Resnest_Melanoma
elif args.enet_type == 'seresnext101':
    ModelClass = Seresnext_Melanoma
elif 'efficientnet' in args.enet_type:
    ModelClass = Effnet_Melanoma
else:
    raise NotImplementedError()

DP = len(os.environ['CUDA_VISIBLE_DEVICES']) > 1

predict()    

100%|██████████| 172/172 [05:44<00:00,  2.00s/it]
